# Imports

In [1]:
import osmnx as ox, networkx as nx, geopandas as gpd, matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon
from descartes import PolygonPatch
import pandas as pd
import numpy as np
ox.config(log_console=True, use_cache=True)

ModuleNotFoundError: No module named 'osmnx'

# Data loads and configurations

In [ ]:
# configure the place, network type, trip times, and travel speed
place = 'San Joaquin County, CA, USA'
trip_times = [1, 3, 5, 10, 15, 20, 25, 30] #in minutes
travel_speed_walk = 4.5 #walking speed in km/hour
travel_speed_bike = 15
travel_speed_car = 40
G_walk = ox.graph_from_place(place, network_type='walk')
G_bike = ox.graph_from_place(place, network_type='bike')
G_car = ox.graph_from_place(place, network_type='drive')

In [ ]:
#Stockton Blocks
BGs = gpd.read_file('blocks/Census2010Block.shp')
BGs = BGs.to_crs(epsg = 4326)

In [ ]:
BGs = BGs[BGs.GEOID10 == '060770022023002']

In [ ]:
#Generate centroids for each block
centroids = BGs['geometry'].centroid
xs = []
ys = []
for centroid in enumerate(centroids):
    xs.append(centroid[1].xy[0])
    ys.append(centroid[1].xy[1])

In [ ]:
# add an edge attribute for time in minutes required to traverse each edge
# WALK
meters_per_minute = travel_speed_walk * 1000 / 60 #km per hour to m per minute
for u, v, k, data in G_walk.edges(data=True, keys=True):
    data['time'] = data['length'] / meters_per_minute

In [ ]:
# add an edge attribute for time in minutes required to traverse each edge
# bike
meters_per_minute = travel_speed_bike * 1000 / 60 #km per hour to m per minute
for u, v, k, data in G_bike.edges(data=True, keys=True):
    data['time'] = data['length'] / meters_per_minute

In [ ]:
# add an edge attribute for time in minutes required to traverse each edge
# drive
meters_per_minute = travel_speed_car * 1000 / 60 #km per hour to m per minute
for u, v, k, data in G_car.edges(data=True, keys=True):
    data['time'] = data['length'] / meters_per_minute

In [ ]:
isochrone_polys = []
center_node = ox.get_nearest_node(G_car, (ys[0][0], xs[0][0]))
for trip_time in sorted(trip_times, reverse=True):
    subgraph = nx.ego_graph(G_car, center_node, radius=trip_time, distance='time')
    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)

# plot the network then add isochrones as colored descartes polygon patches
iso_colors = ox.get_colors(n=len(trip_times), cmap='Reds', start=0.3, return_hex=True)
fig, ax = ox.plot_graph(G_car, fig_height=8, show=False, close=False, edge_color='k', edge_alpha=0.2, node_color='none')
for polygon, fc in zip(isochrone_polys, iso_colors):
    patch = PolygonPatch(polygon, fc=fc, ec='none', alpha=0.6, zorder=-1)
    ax.add_patch(patch)
plt.show()

In [ ]:
isochrone_car = pd.DataFrame(isochrone_polys)
isochrone_car['range'] = [1,3,5,10,15,20,25,30]

In [ ]:
isochrone_car = gpd.GeoDataFrame(isochrone_car, geometry=0)

In [ ]:
isochrone_car['geometry'] = isochrone_car.ix[:,0]

In [ ]:
isochrone_car.range = isochrone_car.range.astype(str)

In [ ]:
isochrone_car = isochrone_car.set_geometry('geometry')
isochrone_car[['geometry','range']].to_file('carIso_1_6_19')

In [ ]:
isochrone_polys = []
center_node = ox.get_nearest_node(G_bike, (ys[0][0], xs[0][0]))
for trip_time in sorted(trip_times, reverse=True):
    subgraph = nx.ego_graph(G_bike, center_node, radius=trip_time, distance='time')
    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)

# plot the network then add isochrones as colored desbiketes polygon patches
iso_colors = ox.get_colors(n=len(trip_times), cmap='Reds', start=0.3, return_hex=True)
fig, ax = ox.plot_graph(G_bike, fig_height=8, show=False, close=False, edge_color='k', edge_alpha=0.2, node_color='none')
for polygon, fc in zip(isochrone_polys, iso_colors):
    patch = PolygonPatch(polygon, fc=fc, ec='none', alpha=0.6, zorder=-1)
    ax.add_patch(patch)
plt.show()

In [ ]:
isochrone_bike = pd.DataFrame(isochrone_polys)
isochrone_bike['range'] = [1,3,5,10,15,20,25,30]

In [ ]:
isochrone_bike = gpd.GeoDataFrame(isochrone_bike, geometry=0)

In [ ]:
isochrone_bike['geometry'] = isochrone_bike.ix[:,0]

In [ ]:
isochrone_bike.range = isochrone_bike.range.astype(str)

In [ ]:
isochrone_bike = isochrone_bike.set_geometry('geometry')
isochrone_bike[['geometry','range']].to_file('bikeIso_1_6_19')

In [ ]:
isochrone_polys = []
center_node = ox.get_nearest_node(G_walk, (ys[0][0], xs[0][0]))
for trip_time in sorted(trip_times, reverse=True):
    subgraph = nx.ego_graph(G_walk, center_node, radius=trip_time, distance='time')
    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)

# plot the network then add isochrones as colored deswalktes polygon patches
iso_colors = ox.get_colors(n=len(trip_times), cmap='Reds', start=0.3, return_hex=True)
fig, ax = ox.plot_graph(G_walk, fig_height=8, show=False, close=False, edge_color='k', edge_alpha=0.2, node_color='none')
for polygon, fc in zip(isochrone_polys, iso_colors):
    patch = PolygonPatch(polygon, fc=fc, ec='none', alpha=0.6, zorder=-1)
    ax.add_patch(patch)
plt.show()

In [ ]:
isochrone_walk = pd.DataFrame(isochrone_polys)
isochrone_walk['range'] = [1,3,5,10,15,20,25,30]

In [ ]:
isochrone_walk = gpd.GeoDataFrame(isochrone_walk, geometry=0)

In [ ]:
isochrone_walk['geometry'] = isochrone_walk.ix[:,0]

In [ ]:
isochrone_walk.range = isochrone_walk.range.astype(str)

In [ ]:
isochrone_walk[['geometry','range']]

In [ ]:
isochrone_walk = isochrone_walk.set_geometry('geometry')

In [ ]:
isochrone_walk[['range', 'geometry']][~isochrone_walk.geometry.type.isin(["LineString", 'Point'])].to_file('walkIso_1_6_19')